## Instalación de requerimientos

In [ ]:
%pip install -r requirements.txt

## Carga y preprocesamiento de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scripts.preprocessing import load_data, preprocess_data

train_path = 'data/ECG5000_train.csv'
test_path = 'data/ECG5000_test.csv'

train_data, test_data = load_data(train_path, test_path) #carga de datos

x_train_scaled, x_test_scaled, scaler = preprocess_data(train_data, test_data) #se escalan los datros de entrenamiento y prueba y se guarda el escalador (MinMaxScaler())


### Visualización de muestras

In [ ]:
from scripts.visualization import plot_ecg_samples

x_train_descaled = scaler.inverse_transform(x_train_scaled)
x_test_descaled = scaler.inverse_transform(x_test_scaled)

# Sin escalar
plot_ecg_samples(x_train_descaled[train_data[0] == 1],
                 x_train_descaled[train_data[0] == 2],
                 x_train_descaled[train_data[0] == 3],
                 x_train_descaled[train_data[0] == 4],
                 x_train_descaled[train_data[0] == 5],
                 ind=7)

# Escalados
# plot_ecg_samples(x_train_scaled[train_data[0] == 1],
#                  x_train_scaled[train_data[0] == 2],
#                  x_train_scaled[train_data[0] == 3],
#                  x_train_scaled[train_data[0] == 4],
#                  x_train_scaled[train_data[0] == 5],
#                  ind=10)

## Entrenamiento del autoencoder

In [ ]:
from scripts.model import build_autoencoder, compile_and_train

# Constructor
input_dim = x_train_scaled[train_data[0] == 1].shape[1] #SOlo tomamos los datos de entrenamiento de la catagorìa '0' (normales)
autoencoder = build_autoencoder(input_dim)

# Entrenamiento del autoencoder
historia = compile_and_train(autoencoder, x_train_scaled[train_data[0] == 1], x_test_scaled[test_data[0] == 1], epochs=30, batch_size=512) 

autoencoder.save_weights('models/autoencoder_weights.weights.h5')

### Visualizaciones de historia y summary

In [ ]:
from scripts.visualization import plot_training_history
# Es necesario haber corrido el bloque anterio para obtener "historia" y "autoencoder"
plot_training_history(historia) 

autoencoder.summary()

### Visualización de reconstrucciones

In [ ]:
from scripts.model import build_autoencoder
from scripts.visualization import plot_ecg_reconstruction

# Constructor del autoencoder
input_dim = x_train_scaled.shape[1]
autoencoder = build_autoencoder(input_dim)

# Carga de los pesos
autoencoder.load_weights('models/autoencoder_weights.weights.h5')

x_test_1_s = x_test_scaled[test_data[0] == 1]
x_test_2_s = x_test_scaled[test_data[0] == 2]
x_test_3_s = x_test_scaled[test_data[0] == 3]
x_test_4_s = x_test_scaled[test_data[0] == 4]
x_test_5_s = x_test_scaled[test_data[0] == 5]

plot_ecg_reconstruction(autoencoder, x_test_1_s, x_test_5_s, index=5)



### Visualización de histograma de distribución

En el siguiente gráfico se muestra la distribución de la función de pérdida al reconstruir (.predict) ECG normales y los 4 tipos de anormales

In [ ]:
from scripts.visualization import load_autoencoder_weights, calculate_losses, plot_loss_histograms
from scripts.model import build_autoencoder

# Construir el autoencoder y cargar los pesos
input_dim = x_train_scaled.shape[1] 
autoencoder = build_autoencoder(input_dim)
autoencoder = load_autoencoder_weights(autoencoder, 'models/autoencoder_weights.weights.h5')

# Calculo de pérdidas
losses = calculate_losses(autoencoder, [x_test_1_s, x_test_2_s, x_test_3_s, x_test_4_s, x_test_5_s])

plot_loss_histograms(losses, threshold=0.08)


In [ ]:
from scripts.visualization import load_autoencoder_weights, calculate_losses, plot_loss_histograms
from scripts.model import build_autoencoder
from scripts.evaluation import calculate_threshold, predict, calculate_sensitivity, calculate_specificity

# Construir el autoencoder y cargar los pesos
input_dim = x_train_scaled.shape[1]
autoencoder = build_autoencoder(input_dim)
autoencoder = load_autoencoder_weights(autoencoder, 'models/autoencoder_weights.weights.h5')

# Calculo las pérdidas
losses = calculate_losses(autoencoder, [x_test_1_s, x_test_2_s, x_test_3_s, x_test_4_s, x_test_5_s])

#threshold = calculate_threshold(losses[0])
threshold = 0.011

print(f"El umbral calculado es: {threshold}")

# Predicciones
preds = [predict(autoencoder, x_test, threshold) for x_test in [x_test_1_s, x_test_2_s, x_test_3_s, x_test_4_s, x_test_5_s]]

# CON RESPECTO A LAS MÉTRICAS, SE UTILIZAN LAS SIGUIENTES TENIENDO EN CUENTA QUE AL SER PREDICCION DE ANOMALIAS CARDIACAS,
# ES MAS IMPORTANTE OBTENER BUENA SENSIBILIDAD, ES DECIR QUE OPTIMICE EL RECONOCIMIENTO DE CASOS ANORMALES A COSTA DE CLASIFICAR ERRONEAMENTE CASOS
# NORMALES COMO ANORMALES


esp_1 = calculate_specificity(preds[0], 'Especificidad (cat. 1, normales)')
sensitivities = [calculate_sensitivity(preds[i], f'Sensitividad (cat. {i+2}, anormales)') for i in range(1, 5)]

plot_loss_histograms(losses, threshold=threshold)
